# MODELING

In [135]:
from tqdm import tqdm
from collections import defaultdict
import gensim
from gensim.corpora import Dictionary
from gensim.models import Phrases
from gensim.models import LdaModel
import pyLDAvis.gensim
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymystem3 import Mystem
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap
import seaborn as sns
%matplotlib inline

# Select the neighborhood and load its data

In [145]:
name = 'Akademicheskij'
district = gpd.read_file('../Data dive/dd2/{}/{}_district.geojson'.format(name,name))
neigh_posts = pd.read_csv('social_media/{}/vk.csv'.format(name))

# Group posts by user

In [147]:
vk_users = pd.DataFrame({'post' : neigh_posts.groupby('userId').apply( lambda x: ' '.join(x['text']))}).reset_index()

# How many users are there?

In [149]:
extra_words = ['http','br','id','com','www', 'instagram', 'vsco', 'https', 'instasize','repost',
              'whatsapp', 'вотсап', 'repostapp','маникюр', 'бровь', 'ресница', 'губа', 'instacollage', 'опубликовывать',
                'фото', 'москва', 'moscow']
def process_docs(docs):
    """
    Function to process texts. Following are the steps we take:
    
    1. Text tokenization.
    2. Removing numbers 
    3. Stopword and short words Removal.
    4. Lemmatization and filter words by their length.
    
    Args:
    ----------
    texts: Tokenized texts.
    
    Returns:
    -------
    texts: Pre-processed tokenized texts.
    """
    m = Mystem()
    # Split the documents into tokens.
    tokenizer = RegexpTokenizer(r'\w+')
    stops = stopwords.words('russian') + stopwords.words('english') + extra_words
    
    for idx in tqdm(range(len(docs))):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not any(c.isdigit() for c in token) and ('id' not in token or 'club' not in token or 'ru' not in token)] for doc in tqdm(docs)]
    #Lemmatize words
    docs = [[m.lemmatize(token)[0] for token in doc ] for doc in tqdm(docs)]
    #Remove stopwords
    docs = [[token for token in doc if token not in stops] for doc in tqdm(docs)]
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in tqdm(docs)]
    return docs
    
def get_corpus(docs):
    
    """Add bigrams to docs and create corpus and dictionary for training
    
    Args:
        docs: list of tokenized and cleaned texts;
    Returns:
        corpus: list of lists of tuples, where first element of tuple is a word id
        and the second is the count of that word in the whole corpus
        dictionary: gensim.corpora.dictionary.Dictionary 
  
    """
    
    frequency = defaultdict(int)
    for text in tqdm(docs):
        for token in text:
            frequency[token] += 1

    texts = [[token for token in text if frequency[token] > 3] for text in tqdm(docs)]

    #Take the bigram, if token is a bigram, add to document.
    bigram = Phrases(texts, min_count = 20)
    for idx in tqdm(range(len(texts))):
        for token in bigram[texts[idx]]:
            if '_' in token:
                texts[idx].append(token)
    
    # Create a dictionary representation of the documents.
    dictionary = Dictionary(texts)
    # Filter out words that occur less than 20 documents, or more than 50% of the documents.
    #dictionary.filter_extremes(no_below=20, no_above=0.5)
    corpus = [dictionary.doc2bow(doc) for doc in tqdm(texts)]
    
    print('Number of unique tokens: {}'.format(len(dictionary)))
    print('Number of documents: {}'.format(len(corpus)))
    
    return corpus, dictionary

# Process texts to be ready for modeling

In [150]:
texts = vk_users['post'].copy()

In [ ]:
docs = process_docs(texts.values)

In [ ]:
corpus, dictionary = get_corpus(docs)

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


#The training model - we use online LDA model which allows to update the model 
#and the following parameters should be defined
num_topics = 10 # number of topics
chunksize = 1000 
passes = 10
iterations = 400
eval_every = 10  #evaluate model perplexity.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha=0.001, update_every = 1, \
                       num_topics=num_topics,\
                       eval_every=eval_every, passes = passes)

In [ ]:
# import nltk
# nltk.download()

In [ ]:
data = pyLDAvis.gensim.prepare(model, corpus, dictionary); # visualize lda topics
pyLDAvis.display(data)

# Assign topics to users

In [ ]:
docTopicProbMat = model[corpus]
lda_users = vk_users.copy()
lda_users['topics'] = docTopicProbMat
vk_users['topic'] = lda_users['topics'].apply(lambda x :x[0][0])

# Assign topics to their posts

In [227]:
neigh_posts['topic'] = neigh_posts['userId'].apply(lambda userId: vk_users.loc[vk_users['userId']==userId,'topic'].item())

# Plot the histogram of topic distribution per posts and per users

# Mapping

In [233]:
center_lat = list(district.centroid[0].coords)[0][1]
center_lon = list(district.centroid[0].coords)[0][0]

In [ ]:
map_places = folium.Map([center_lat, center_lon], tiles='Stamen Toner', zoom_start=14,control_scale=True)

#Define style for geojson objects
style_function = lambda feature: dict(fillColor='#AECCAE',
                                      color='#AECCAE',
                                      weight=1,
                                      opacity=0.3)

# Adding Houses
houses = gpd.read_file('../Data dive/dd2/{}/{}_chruchevki.geogson'.format(name, name))
points = folium.features.GeoJson(houses,name='Khurshevki houses')
map_places.add_child(points,name='Khurshevki houses')

#Adding district
polygon = folium.features.GeoJson(district, style_function=style_function,name='district boundary')
map_places.add_child(polygon,name='district boundary')

colormap_dict = {0.0: 'pink', 0.3: 'blue', 0.5: 'green',  1.0: 'red'}

#Adding topics heatmaps
for topic_id in [0, 1, 2]:
    topic_coords = list(zip(neigh_posts[neigh_posts.topic == topic_id].lat, neigh_posts[neigh_posts.topic == topic_id].lon))

    HeatMap(topic_coords,
            name='Topic: {}'.format(topic_id),
            radius=10, 
            min_opacity=0.8,
            gradient={0.0: 'pink', 0.3: 'blue', 0.5: 'green',  1.0: 'red'}).add_to(map_places)


    colormap = folium.LinearColormap(colors = colormap_dict.values())
    colormap.caption = 'Topic: {}'.format(topic_id)

#Switch between layers
folium.LayerControl().add_to(map_places)
map_places